In [1]:
# Standard libraries
import sys
import math
import random
import time
import os

# Third-party imports
import numpy as np
import pandas as pd
import pickle as pkl

import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

import keras_tuner as kt

import importlib
from yapf.yapflib.yapf_api import FormatCode
import GPUtil

# Project imports
import ephesus
import utils
from const import *

#### GPU Setup

In [2]:
# Specific GPUs
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[2:4], 'GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
mirro_strat = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


#### Data Preprocessing

In [3]:
# Find the most recent file in injec_curve
datas = utils.retur_most_recen(f'{xom_data_path}padde_cuts/')
# datas = 'padde_cuts-34745-2850-6998.pkl'
print(f'Most recent file: {datas}')

with open(f'{xom_data_path}padde_cuts/{datas}','rb') as f:
    raw_x_data = pkl.load(f)

Most recent file: combi-padde_cuts-345235-30352-72662.pkl


In [4]:
# Dataset composition setup
DETEC_TYPE = 'plane_moon_cut_injec'
NUMBE_RELEV = 22000
NUMBE_IRREL = 28000

In [5]:
# Separate full dataset into relevant and irrelevant components
full_relev_index = []
for i in range(len(raw_x_data)):
    if raw_x_data[i, -1, 1][DETEC_TYPE]:
        full_relev_index.append(i)
        
# Take a random sample of a relevant
relev_index = random.sample(full_relev_index, NUMBE_RELEV)
relev_curve = np.copy(raw_x_data[relev_index])
# Take a random sample of a irrelevant
irrel_curve = np.delete(raw_x_data, full_relev_index, axis=0)
irrel_curve = irrel_curve[utils.retur_rando_sampl(NUMBE_IRREL, len(irrel_curve))]

# Shuffle the order of the data
rando_shuff_x_data = np.concatenate((relev_curve, irrel_curve), axis=0)
np.random.shuffle(rando_shuff_x_data)

# Normalize the data from 0 to 1
norma_full_x_data = np.copy(rando_shuff_x_data)
x_data = np.copy(rando_shuff_x_data[:, :-1, 1].astype(float))

for i in range(len(x_data)):
    chang_slots = np.where(x_data[i] != 0)[0]
    x_data[i, chang_slots] = utils.norma_data(x_data[i, chang_slots])
    norma_full_x_data[i, :-1, 1] = x_data[i]

x_data = np.expand_dims(x_data, 2)

# Create the corresponding y dataset
y_data = np.zeros(len(norma_full_x_data))
for i in range(len(norma_full_x_data)):
    y_data[i] = norma_full_x_data[i, -1, 1][DETEC_TYPE]

print(
    f'{sum(y_data)/len(y_data):.2%} of the dataset contains a full injection')

50.00% of the dataset contains a full injection


In [6]:
# Split the data into training and testing

stop = len(x_data) - len(y_data) % 100
split = int(stop * 0.7)
    
x_train = np.array(x_data[:split])
full_x_train = norma_full_x_data[:split]
x_test = np.array(x_data[split:stop])
full_x_test = norma_full_x_data[split:stop]
y_train = y_data[:split]
y_test = y_data[split:stop]

In [7]:
# Remove non-full batches (because they cause problems)
BATCH_SIZE = 100

# If no remainder, skip
if x_train.shape[0] % BATCH_SIZE:
    x_train = x_train[:-(x_train.shape[0] % BATCH_SIZE)]
    full_x_train = full_x_train[:-(full_x_train.shape[0] % BATCH_SIZE)]
    y_train = y_train[:-(y_train.shape[0] % BATCH_SIZE)]

if x_test.shape[0] % BATCH_SIZE:
    x_test = x_test[:-(x_test.shape[0] % BATCH_SIZE)]
    full_x_test = full_x_test[:-(full_x_test.shape[0] % BATCH_SIZE)]
    y_test = y_test[:-(y_test.shape[0] % BATCH_SIZE)]

In [8]:
# Baseline calculations for email updates
# Baseline accuracy, precision, and recall are all the same
basel_value = sum(y_test) / len(y_test) 
if basel_value < 0.5:
    basel_value = 1 - basel_value

#### Training

In [9]:
# Number of models
NUMBE_OF_MODEL = 80
# Number of executions per model
EXECU_PER_MODEL = 1
# Max number of epochs
EPOCH = 25
# Early stopping patience
PATIE = 4

In [16]:
# Callbacks

tensorboard = TensorBoard(log_dir=f'{main_path}tensorboard/{datas}_{int(time.time())}')
early_stopp = EarlyStopping(monitor='val_accuracy', patience=PATIE)
resto_valid_accur = utils.resto_best_valid_accur()
email_train_progr = utils.email_train_progr()

callb = [resto_valid_accur, early_stopp, email_train_progr]#, tensorboard]

In [17]:
log_direc = f'{main_path}keras_tuner/{datas}_{int(time.time())}'
def tuner_model(hyper):
    '''RNN tuner model for use with keras tuner.'''

    # Setup for model architecture
    MAX_RNN_LAYER = 6
    MAX_DENSE_LAYER = 6
    dense_activ = ['elu', 'tanh']
    metri = ['accuracy', utils.preci, utils.recal]
    rnn_type = hyper.Choice('rnn_type', ['lstm', 'gru'])

    model = keras.Sequential()
    # Add masking layer
    model.add(layers.Masking(mask_value=0.0,input_shape=(1900, 1)))
    
    if rnn_type == 'gru'  or initi:  
        with hyper.conditional_scope('rnn_type', 'gru'):
            # Determine the number of GRU layers
            try:
                numbe_gru_layer = hyper.Int(f'numbe_gru_layer', 1, MAX_RNN_LAYER)
                final_gru_units = hyper.Int(f'final_gru_units', 32, 256)
            except (TypeError, KeyError):
                pass
            if initi:
                numbe_gru_layer = MAX_RNN_LAYER
                final_gru_units = 0
            for curre_gru_layer_numbe in range(1, numbe_gru_layer):
                with hyper.conditional_scope('numbe_gru_layer', list(range(curre_gru_layer_numbe, numbe_gru_layer + 1))):
                    try:
                        model.add(layers.GRU(hyper.Int(f'gru_{curre_gru_layer_numbe}_units', 16, 256), 
                                               activation='tanh', return_sequences=True))
                    except (TypeError, KeyError):
                        pass               
         
    if rnn_type == 'lstm' or initi:  
        with hyper.conditional_scope('rnn_type', 'lstm'):
            # Determine the number of lstm layers
            try:
                numbe_lstm_layer = hyper.Int(f'numbe_lstm_layer', 1, MAX_RNN_LAYER)
                final_lstm_units = hyper.Int(f'final_lstm_units', 32, 256)
            except (TypeError, KeyError):
                pass
            if initi:
                numbe_lstm_layer = MAX_RNN_LAYER
                final_lstm_units = 0
            for curre_lstm_layer_numbe in range(1, numbe_lstm_layer):
                with hyper.conditional_scope('numbe_lstm_layer', list(range(curre_lstm_layer_numbe, numbe_lstm_layer + 1))):
                    try:
                        model.add(layers.LSTM(hyper.Int(f'lstm_{curre_lstm_layer_numbe}_units',16, 256), activation='tanh', return_sequences=True))
                    except (TypeError, KeyError) as e:
                        pass
                        
    if rnn_type == 'gru':
        model.add(layers.GRU(final_gru_units, activation='tanh'))
    elif rnn_type == 'lstm':
        model.add(layers.LSTM(final_lstm_units, activation='tanh')) 
        
    # Determine the number of rnn and dense layers
    numbe_dense_layer = hyper.Int(f'numbe_dense_layer', 1, MAX_DENSE_LAYER)
    if initi:
        numbe_dense_layer = MAX_DENSE_LAYER
        
    # Add the dense and dropout layers
    for curre_dense_layer_numbe in range(1, numbe_dense_layer):
        with hyper.conditional_scope('numbe_dense_layer', list(range(curre_dense_layer_numbe, numbe_dense_layer + 1))):
            try:
                model.add(layers.Dense(hyper.Int(f'dense_{curre_dense_layer_numbe}_units', 16, 256), 
                                       activation=hyper.Choice(f'dense_{curre_dense_layer_numbe}_activ', dense_activ)))
                model.add(layers.Dropout(hyper.Float(f'dense_{curre_dense_layer_numbe}_dropo', 0, 1)))
            except (TypeError, KeyError):
                pass
        
    # Add the sigmoid activation for binary classification
    model.add(layers.Dense(1, activation='sigmoid'))
        
    # Setup the learning and decay rate
    learn_rate = hyper.Float('learn_rate', 1e-6, 1e-2)
    decay_rate = hyper.Float('decay_rate', 1e-6, 1e-2)

    # Create the optimizer (always Nadam)
    optim = tf.keras.optimizers.Nadam(learning_rate=learn_rate,
                                    decay=decay_rate)
    # Compile the model
    model.compile(loss='BinaryCrossentropy',
                  optimizer=optim,
                  metrics=metri)

    return model

### all the try except typeerror statemetns are added cause initial values are none

In [18]:
utils.tuner_messa_backg_infor(setup=True, total_trial_numbe=(NUMBE_OF_MODEL*EXECU_PER_MODEL), total_epoch_numbe=EPOCH,
                              basel_accur=basel_value, basel_preci=basel_value, basel_recal=basel_value)

In [19]:
initi = True
tuner = kt.BayesianOptimization(hypermodel=tuner_model,
                                objective='val_accuracy',
                                max_trials=NUMBE_OF_MODEL,
                                executions_per_trial=EXECU_PER_MODEL,
                                distribution_strategy=mirro_strat,
                                directory=log_direc)

In [20]:
initi = False
try:
    tuner.search(x_train,
                 y_train,
                 verbose=2,
                 epochs=EPOCH,
                 batch_size=BATCH_SIZE,
                 callbacks=callb,
                 validation_data=(x_test, y_test))
except Exception as excep:
    utils.send_task_comple_email('Trial Error', str(excep))
    raise excep

Trial 1 Complete [00h 04m 05s]
val_accuracy: 0.5400000214576721

Best val_accuracy So Far: 0.5400000214576721
Total elapsed time: 00h 04m 05s

Search: Running Trial #2

Hyperparameter    |Value             |Best Value So Far 
rnn_type          |gru               |gru               
numbe_gru_layer   |2                 |6                 
final_gru_units   |159               |87                
gru_1_units       |186               |20                
gru_2_units       |111               |189               
numbe_dense_layer |3                 |5                 
dense_1_units     |157               |201               
dense_1_activ     |elu               |elu               
dense_1_dropo     |0.74111           |0.91512           
dense_2_units     |188               |242               
dense_2_activ     |tanh              |elu               
dense_3_units     |201               |103               
dense_3_activ     |elu               |elu               
learn_rate        |0.001809      

KeyboardInterrupt: 

In [ ]:
utils.send_task_comple_email('KT')

In [ ]:
model = tuner.get_best_models(num_models=5)

In [ ]:
model_num = 0
for model_num in range(len(model)):
    model_path = f'{xom_data_path}plane_moon_model/{datas}-{model_num}-{int(time.time())}.h5'
    model[model_num].save(model_path)
    print(f'Best Model Path: {model_path}')

## Testing

#### Formatting